In [1]:
import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import SGD
import cv2
import skimage
import os
from imgaug.imgaug import augmenters as iaa
from densenet121 import DenseNet
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

from sklearn.cluster import MiniBatchKMeans

Using TensorFlow backend.


In [2]:
model = DenseNet(reduction=0.5, classes=1000, weights_path='./densenet121_weights_tf.h5')
print(model.layers[-1].output_shape)
model.layers.pop()
model.layers.pop()
output = model.layers[-1].output
new_model = Model(model.input, output)

densenet121.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (7, 7), strides=(2, 2), use_bias=False, name="conv1")`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv2_1_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv2_1_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:164: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))
/home/wanlu/4032/venv/local/lib/pyt

densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv3_12_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:135: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), use_bias=False, name="conv3_blk")`
  x = Convolution2D(int(nb_filter * compression), 1, 1, name=conv_name_base, bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_1_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_1_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4

densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_21_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_22_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_22_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_23_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_23_x2

(None, 1000)


In [3]:
new_model.layers[-1].output_shape

(None, 1024)

In [4]:
# train_features.shape

In [32]:
def get_data(features_set, labels_set):
    print "get_data"
    group_element_num = 100
    
    label_to_features = {}
    label_to_indexes = {}
    for index, label in enumerate(labels_set):
        label = tuple(label)
        if label not in label_to_features:
            label_to_features[label] = [features_set[index]]
            label_to_indexes[label] = [index]
        else:
            label_to_features[label].append(features_set[index])
            label_to_indexes[label].append(index)

    labels = []
    img_features = []
    indexes = []
    for l in label_to_features.keys():
        if len(label_to_features[l]) % group_element_num == 0:
            split = len(label_to_features[l])/group_element_num
        else:
            split = len(label_to_features[l])/group_element_num + 1
        for i in range(split):
            labels.append(l)
            # if it's the last part
            if i == split-1:
                img_features.append(np.asarray(label_to_features[l][i*group_element_num:]))
                indexes.append(np.asarray(label_to_indexes[l][i*group_element_num:]))
            # split into parts, each with 'group_element_num' img featrues
            else:
                img_features.append(np.asarray(label_to_features[l][i*group_element_num:(i+1)*group_element_num]))
                indexes.append(np.asarray(label_to_indexes[l][i*group_element_num:(i+1)*group_element_num]))

    labels = np.asarray(labels)
    img_features = np.asarray(img_features)
    indexes = np.asarray(indexes)
    
    return my_shuffle([img_features, labels, indexes])

In [6]:
def train(iteration, train_X, train_y):
    print "training"
    # classifier = OneVsRestClassifier(LinearSVC(random_state=0))
    lst_clfs = [LinearSVC() for i in range(9)]
    for i in range(iteration):
        print "Iteration: " + str(i)
        for k in range(9):
            clf = lst_clfs[k]
            part = train_X.shape[0]/5
            scores = []
            for fold in range(5):
                val_X = train_X[fold*part:(fold+1)*part]
                val_y = train_y[fold*part:(fold+1)*part]
                tr_X = np.vstack((train_X[0:fold*part], train_X[(fold+1)*part:]))
                tr_y = np.vstack((train_y[0:fold*part], train_y[(fold+1)*part:]))

                clf.fit(tr_X, tr_y[:, k])
                val_score = clf.score(val_X, val_y[:, k])
                scores.append(val_score)
            print("Training accuracy: %0.2f (+/- %0.2f)" % (np.mean(scores), np.std(scores) * 2))
            
    return lst_clfs

In [7]:
def test(lst_clfs, test_X, test_y):
    for k in range(9):
        clf = lst_clfs[k]
        s = clf.score(test_X, test_y[:,k])
        print str(k) + "'s classifier, test acc = " + str(round(s, 4))
    

In [8]:
def get_biz_features(img_features, labels, indexes, num_cluster, kmn):
    biz_features = np.zeros([len(labels),(1024*num_cluster)])

    for idx, features in enumerate(img_features):
        feature_index = indexes[idx]
        cluster_lable = np.array(kmn[feature_index])
        
        # for each biz group, mean feature vectore for those in the same cluster
        for kn in range(num_cluster):
            x = features[cluster_lable==kn]
            # if feature doesn't belong to any cluster,
            # which is impossible....
            if(len(x) == 0):    
                biz_features[idx,(1024*(kn)):(1024*(kn+1))] = np.zeros([1,1024])
            else:
                x = np.mean(x,axis=0)
                x = x.reshape([1,1024])
                biz_features[idx,(1024*(kn)):(1024*(kn+1))] = x

    return biz_features

In [9]:
def my_shuffle(arr_list):
    s_index = np.arange(len(arr_list[0]))
    np.random.shuffle(s_index)
    return_lst = []
    for arr in arr_list:
        return_lst.append(arr[s_index])
    return tuple(return_lst)

In [10]:
input_features = np.load('../data/features_234000.npy')
input_labels = np.load('../data/features_labels_234000.npy')

In [31]:
for num_cluster in [2,3,4,5]:
    print "num_cluster = " + str(num_cluster)
    # shuffle input data
#     input_features, input_labels = shuffle(input_features, input_labels, random_state=7)
    
    input_features, input_labels = my_shuffle([input_features, input_labels])
    
    kmn_holder = MiniBatchKMeans(n_clusters=num_cluster)
    kmn = kmn_holder.fit_predict(input_features, input_labels)
    kmn_train = kmn[:1880000]
    kmn_test = kmn[188000:]
    
    train_data = input_features[:188000]
    train_label = input_labels[:188000]
    
    test_data = input_features[188000:]
    test_label = input_labels[188000:]
    
    tr_img_features, tr_labels, tr_indexes = get_data(train_data, train_label)
    tr_biz_features = get_biz_features(tr_img_features, tr_labels, tr_indexes, num_cluster, kmn_train)

    ts_img_features, ts_labels, ts_indexes = get_data(test_data,test_label)
    ts_biz_features = get_biz_features(ts_img_features, ts_labels, ts_indexes, num_cluster, kmn_test)

    #     for idx, features in enumerate(img_features):
#         feature_index = indexes[idx]
#         cluster_lable = np.array(kmn_train[feature_index])
        
#         # for each biz group, mean feature vectore for those in the same cluster
#         for kn in range(num_cluster):
#             x = features[cluster_lable==kn]
#             # if feature doesn't belong to any cluster,
#             # which is impossible....
#             if(len(x) == 0):    
#                 biz_features[idx,(1024*(kn)):(1024*(kn+1))] = np.zeros([1,1024])
#             else:
#                 x = np.mean(x,axis=0)
#                 x = x.reshape([1,1024])
#                 biz_features[idx,(1024*(kn)):(1024*(kn+1))] = x
    
    one_vs_rest_train_test(tr_biz_features, tr_labels, ts_biz_features, ts_labels)
#     list_of_classifiers = train(1, tr_biz_features, tr_labels)
    
#     test(list_of_classifiers, ts_biz_features, ts_labels)

num_cluster = 2
get_data
124
get_data
124
test
F1 score:  0.997791608111
Individual Class F1 score:  [ 0.992       0.9968254   0.99696049  0.99626866  1.          0.99737533
  1.          0.99746835  1.        ]
num_cluster = 3
get_data
124
get_data
123
test
F1 score:  0.997981429148
Individual Class F1 score:  [ 0.99598394  1.          0.99846861  0.99621212  0.99767981  1.
  0.99874529  0.99746835  0.99415205]
num_cluster = 4
get_data
124
get_data
123
test
F1 score:  0.998185117967
Individual Class F1 score:  [ 1.          0.99683544  0.99693252  0.99809524  0.99767981  0.998679
  0.99874844  0.99749373  1.        ]
num_cluster = 5
get_data
124
get_data
124
test
F1 score:  0.996581540318
Individual Class F1 score:  [ 0.98387097  0.99680511  0.99540582  0.99437148  1.          0.99606815
  1.          0.99749373  0.99806576]


In [12]:
tr_biz_features.shape

(1928, 2048)

In [29]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
def one_vs_rest_train_test(train_X, train_y, test_X, test_y):
    
    # classifier = OneVsRestClassifier(SVC(kernel='linear', probability=True))
    classifier = OneVsRestClassifier(LinearSVC(loss='hinge'))
    classifier.fit(train_X, train_y)
    
    from sklearn.metrics import f1_score
    print "test"
    y_predict = classifier.predict(test_X)
    print "F1 score: ", f1_score(test_y, y_predict, average='micro')
    print "Individual Class F1 score: ", f1_score(test_y, y_predict, average=None)

In [ ]:
# from sklearn.metrics import accuracy_score
# clf = LinearSVC()
# clf.fit(input_features[:200000], input_labels[:200000, 0])
# y_pred = clf.predict(input_features[200000:])
# accuracy_score(input_labels[200000:,0], y_pred)

In [ ]:
# from sklearn.metrics import f1_score
# clf = LinearSVC()
# clf.fit(tr_biz_features, tr_labels[:, 0])
# y_pred = clf.predict(ts_biz_features)
# print "F1 score: ", f1_score(ts_labels[:,0], y_pred) 
# print "Individual Class F1 score: ", f1_score(y_ptest, y_ppredict, average=None)

In [ ]:
ts_labels.shape

In [ ]:
ones = zeros = 0
for l in ts_labels:
    print "each label"
    ones += np.sum(l)
    print np.sum(l)
    zeros += (len(l) - np.sum(l))
    print (len(l) - np.sum(l))

In [ ]:
ones

In [ ]:
ts_labels[:20]